# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /home/anna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/anna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# import libraries
import pandas as pd
import re
import numpy as np
from time import time
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeClassifier
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [4]:
# load data from database
def load_data():
    """Function reads sql database into pandas dataframe and splits into 
    input and output variables
    Args:
    None

    Returns:
    X -> input variable
    y -> output variables
    """ 
    engine = create_engine('sqlite:///InsertDatabaseName.db')
    df = pd.read_sql("SELECT * FROM InsertTableName", engine)
    X = df['message']
    y = df[df.columns[4:]]
    
    return X, y
X,y = load_data()


In [5]:
labels = list(y.columns)
len(labels)

36

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [7]:
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
# Define a pipeline combining a text feature extractor with a MultiOutputClassifier
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf' , MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)
# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
# calculate and print classification performance for each category
print(classification_report(y_test.values,y_pred,target_names =labels))

                                 precision    recall  f1-score   support

               related_category       0.81      0.97      0.89      4984
               request_category       0.90      0.44      0.59      1135
                 offer_category       0.00      0.00      0.00        21
           aid_related_category       0.77      0.64      0.70      2690
          medical_help_category       0.59      0.06      0.10       518
      medical_products_category       0.71      0.05      0.10       322
     search_and_rescue_category       1.00      0.04      0.07       181
              security_category       0.40      0.02      0.03       121
              military_category       0.73      0.03      0.07       231
           child_alone_category       0.00      0.00      0.00         0
                 water_category       0.87      0.28      0.42       435
                  food_category       0.88      0.46      0.60       747
               shelter_category       0.87      0.

/home/anna/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anna/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anna/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anna/.local/lib/python3.7/site-packages/sklearn/

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
# check available parameters to tune
pipeine_parms = pipeline.get_params()
#pipeine_parms

In [11]:
parameters = {
    #'vect__max_df': (0.5, 0.75),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    #'clf__estimator__n_estimators': [10]
   
}
#find the best parameters for both the feature extraction and the
# classifier
cv = GridSearchCV(estimator=pipeline, param_grid=parameters, 
                  cv=3,n_jobs=-1,verbose=10)

In [12]:
print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
print(parameters)
t0 = time()

# train classifier
cv.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()
print("Best score: %0.3f" % cv.best_score_)
print("Best parameters set:")
best_parameters = cv.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'vect__ngram_range': ((1, 1), (1, 2)), 'tfidf__norm': ('l1', 'l2')}
Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:  4.9min remaining: 14.6min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:  5.0min remaining:  7.0min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed: 15.6min remaining: 11.1min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed: 15.8min remaining:  5.3min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 16.2min finished


done in 1412.354s

Best score: 0.248
Best parameters set:
	tfidf__norm: 'l2'
	vect__ngram_range: (1, 1)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
# predict on test data
y_pred =cv.predict(X_test)

# calculate and print classification performance for each category
print(classification_report(y_test.values,y_pred,target_names =labels))

                                 precision    recall  f1-score   support

               related_category       0.83      0.93      0.88      5041
               request_category       0.85      0.40      0.55      1117
                 offer_category       0.00      0.00      0.00        28
           aid_related_category       0.78      0.48      0.60      2732
          medical_help_category       0.64      0.07      0.13       533
      medical_products_category       0.61      0.05      0.10       317
     search_and_rescue_category       0.80      0.02      0.04       188
              security_category       0.33      0.01      0.02       117
              military_category       0.57      0.04      0.07       216
           child_alone_category       0.00      0.00      0.00         0
                 water_category       0.91      0.27      0.41       433
                  food_category       0.85      0.31      0.45       720
               shelter_category       0.88      0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [24]:
#calculate the precision and accuracy score

print("Precision score: {}".format(precision_score(y_test.values,y_pred,
                                                  average='weighted')))
print("Accuracy score:{}".format(metrics.accuracy_score(y_test.values,y_pred)))

Precision score: 0.7741858730679443
Accuracy score:0.3994486138765508


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
vectorizers = [CountVectorizer(tokenizer=tokenize),HashingVectorizer(tokenizer=tokenize)]
classifiers = [KNeighborsClassifier(n_neighbors=36),
               RandomForestClassifier()]
for v in vectorizers:
    t0 = time()
    print('_' * 80)
    print("Training: ")
    print("Vectorizer: ",type(v).__name__)
    #print(type(v).__name__)
    print()
    for c in classifiers:
        print("Classifier: ",type(c).__name__)
        print()
        pipeline = Pipeline([
                  ('vect', v),
                  ('tfidf', TfidfTransformer()),
                  ('clf' , c)
                    ])
        # train classifier
        pipeline.fit(X_train, y_train)

        # predict on test data
        y_pred = pipeline.predict(X_test)
        train_time = time() - t0
        print("train time: %0.3fs" % train_time)
        score = precision_score(y_test[y_test.columns[1:]].values,y_pred[:, 1:],average='weighted')
        print("precision:   %0.3f" % score)    
        print()

________________________________________________________________________________
Training: 
Vectorizer:  CountVectorizer

Classifier:  KNeighborsClassifier

train time: 24.652s
precision:   0.664

Classifier:  RandomForestClassifier



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


train time: 51.317s
precision:   0.702

________________________________________________________________________________
Training: 
Vectorizer:  HashingVectorizer

Classifier:  KNeighborsClassifier



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


train time: 23.918s
precision:   0.664

Classifier:  RandomForestClassifier



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


train time: 348.953s
precision:   0.711



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
pipeline2 = Pipeline([
        ('vect', HashingVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf' , MultiOutputClassifier(RandomForestClassifier()))
    ])

In [30]:
# train classifier
pipeline2.fit(X_train, y_train)

# predict on test data
y_pred = pipeline2.predict(X_test)

In [31]:
# calculate and print classification performance for each category
print(classification_report(y_test[y_test.columns[1:]].values,y_pred[:, 1:],target_names =labels[1:]))

                                 precision    recall  f1-score   support

               request_category       0.82      0.30      0.44      1117
                 offer_category       0.00      0.00      0.00        28
           aid_related_category       0.75      0.44      0.55      2732
          medical_help_category       0.64      0.03      0.06       533
      medical_products_category       0.67      0.02      0.04       317
     search_and_rescue_category       1.00      0.02      0.03       188
              security_category       0.33      0.01      0.02       117
              military_category       0.00      0.00      0.00       216
           child_alone_category       0.00      0.00      0.00         0
                 water_category       0.92      0.13      0.22       433
                  food_category       0.88      0.18      0.29       720
               shelter_category       0.89      0.10      0.18       556
              clothing_category       0.67      0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [ ]:
#pickle.dump(cv, open('model.pickle', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.